LLM prompt for grade

## Lib import

In [ ]:
import os
import logging
from typing import List, Literal, Annotated, Optional, Union
from typing_extensions import TypedDict
import chromadb
from chromadb.config import Settings

from langchain import PromptTemplate, LLMChain
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.runnables import RunnableSerializable
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_chroma import Chroma
from langchain_text_splitters import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.schema import Document
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_huggingface import HuggingFaceEmbeddings

from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph
from langgraph.checkpoint.memory import InMemorySaver

## Init and API key

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# key
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
silicon_api_key = os.getenv("SILICON_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

# deepseek
deepseek_llm_model = "deepseek-chat"

# silicon
silicon_base_url =  "https://api.siliconflow.cn/v1"
silicon_llm_model = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# huggingface
huggingface_embed_model = "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
logging.basicConfig(level=logging.INFO)

## LLM Init

In [ ]:
# init LLM mod
llm_deepseek = ChatDeepSeek(
    model=deepseek_llm_model,
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    top_p=0.9,
    frequency_penalty=0.7,
    presence_penalty=0.5,
    max_retries=3
)

## Deepseek Relevance Grader

In [ ]:
################################################################################
###  Document Relevance Grader
################################################################################
class DocumentRelevanceGrader:
    """
    Document Relevance Grader.
    Binary score for relevance check on retrieved documents.
    """

    def __init__(
        self,
        llm: RunnableSerializable,
        system_prompt: Optional[str] = None,
        grade_schema: Optional[BaseModel] = None
    ):
        self.grade_schema = grade_schema or self._default_grade_schema()
        self.system_prompt = system_prompt or self._default_system_prompt()
        self.chain = self._build_chain(llm)
    
    @staticmethod
    def _default_grade_schema():
        score_min = 0
        score_max = 10
        class GradeSchema(BaseModel):
            relevance_score: int = Field(
                default=0,
                description=(
                    """
                    \rThe relevance of the document to the question, score between 0-10
                    """
                ),
                examples=[3, 7, 9],
                ge=score_min,
                le=score_max
            )
            
            @validator('relevance_score', pre=True)
            def cast_to_int(cls, value):
                return int(float(value))

            @validator('relevance_score')
            def validate_range(cls, value):
                if value < score_min:
                    value = score_min
                elif value > score_max:
                    value = score_max;
                    
                return value

        return GradeSchema
    
    @staticmethod
    def _default_system_prompt():
        role = """
            \rRole: Document Relevance Scoring Expert
            \rTask: Assess the relevance of the document to the question
            \r
            \rGuidelines:
            \rYou are a professional document relevance scoring expert. Please assess the relevance of the document to the question based on the following criteria:
            \rScoring Criteria:
            \r0 points - Document content is completely irrelevant to the question
            \r3 points - Contains only individual keywords but no real relevance
            \r5 points - Contains relevant keywords but lacks semantic relevance
            \r7 points - Largely relevant but insufficient supporting evidence
            \r9 points - Highly relevant with detailed evidence
            \r10 points - Perfectly matches the problem and provides a complete solution
        """

        respond_request = """
            \rRespond to the request:
            \r1. degree of keyword match
            \r2. semantic relevance
            \r3. strength of evidential support
            \r
            \rCurrent Scenario: Rapid Response Mode
        """

        response_format = """
            \rPlease return a whole number score from 0-10.
            \rNo explanations or additional text.
        """
    
        return f"{role}\n{respond_request}\n{response_format}"
    
    def _build_chain(self, llm: RunnableSerializable) -> RunnableSerializable:
        structured_llm = llm.with_structured_output(self.grade_schema)
        
        prompt_template = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            ("human", "Retrieved document: \n\n {document} \n\n User question: {question}")
        ])
        
        return prompt_template | structured_llm
    
    def grade(
        self,
        document: str,
        question: str,
        **kwargs
    ) -> Union[str, dict]:
        inputs = {
            "document": document,
            "question": question,
            **kwargs
        }
        return self.chain.invoke(inputs)

## Test Grader

In [ ]:
grader = DocumentRelevanceGrader(llm=llm_deepseek)
print(f"system_prompt is {grader.system_prompt}")

In [ ]:
##
## Test Grade: yes
##

user_input = "What are the ingredients in Alpha Hope?"
retrieval_document = """
Alpha Hope has been formulated with two powerful active ingredients, PQQ and Molecular Hydrogen.
They work synergistically to activate metabolic pathways involved in energy production and cognition.
This is particularly formulated to promote the bodyâ\x80\x99s natural detox process and help the body naturally produce Hope Molecules,\
also known as PGC-1Î±, that fight oxidative damage.
"""
grade = grader.grade(question=user_input, document=retrieval_document)
print(f"grade is {grade.relevance_score}")

In [ ]:
##
## Test Grade: no
##

user_input = "What are the ingredients in Alpha Hope?"
retrieval_document = """
Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager, possibly rendering your system unusable.
It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv. Use the --root-user-action option if you know what you are doing and want to suppress this warning.
"""

grade = grader.grade(question=user_input, document=retrieval_document)
print(f"grade is {grade.relevance_score}")